# Street view extraction for use in SfM
This notebook uses the google maps and street view APIs to download street view images between two points with location information identified, and map the resulting points.  The images extracted can be used for Structure from Motion terrain modeling, and the image coordinates can be used as ground control points to georeference the model.

In [9]:
# Import the libraries that we will use in this notebook:
import gmaps, math, urllib, os, json, google_streetview.api
api_key = "your api key"    # Set the API key
gmaps.configure(api_key=api_key)                       # Configure gmaps to use our API key
DownLoc = r'C:\Users\Cemodo\Documents\Python_projects\Coulee_street_view\downloads' # location to save files

# First, we set a fwe beginning parameters:
start_location = [47.829407, -119.133617]
end_location = [47.817773, -119.151449]

# make a guess of the approximate bearing to the next point on the road
rough_direction = 225  # !!not bearing, but direction in degrees but with horizontal origin,
# increasing counter-clockwise like in trig!!
# (my head almost explode thinking about how to convert from map bearing to trig direction... 
# I gave up because it really didn't matter, we just need to pick something that works to get out next point.)
# (I'm sure someone has already figured it out on the internet somewhere... but I didn't go find it.)
# cheat sheet: 0=East, 90=North, 180=West, 270=South
approx_disp = 50 # distance in feet, approximately, between street view images.
heading = 120  # bearing we want to look, holding this constant because the cliff is basically linear and constant.

print('done')

done


In [10]:
# so to do this, we need some maths...  and trigonometry- so pulling out the old SOHCAHTOA!
deg_to_mi = 69 # at this latitude, there is roughly 69 miles to a degree.  This is approximate and should be fine here.
mi_to_ft = 5280 # conversion factor from miles to feet.
disp_deg = round(approx_disp / (mi_to_ft * deg_to_mi),7) # Calculate the approximate displacement in degrees 
y_disp = round(math.sin(math.pi * rough_direction /180)*disp_deg,7) # Calculate the y component of displacement
x_disp = round(math.cos(math.pi * rough_direction /180)*disp_deg,7) # Calculate the x component of displacement
check = round(math.sqrt((x_disp*x_disp)+(y_disp*y_disp)),7)         # Check our math: a^2 + b^2 = c^2
print("approx displacement in degrees is: "+str(disp_deg))
print('x disp = ' + str(x_disp) + '... y disp = ' + str(y_disp))
print('math check = '+str(check))

approx displacement in degrees is: 0.0001372
x disp = -9.7e-05... y disp = -9.7e-05
math check = 0.0001372


In [11]:
# Now let's plot our starting location and end location and view the line we want to extract photos from on a map
fig = gmaps.figure()
extraction_route = gmaps.directions_layer(start_location, end_location)
fig.add_layer(extraction_route)
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# The functions below were copied and modified from here: 
# https://andrewpwheeler.com/2018/04/02/drawing-google-streetview-images-down-an-entire-street-using-python/
key = "&key=" + api_key  # text API key for using with the street view API
# define functions...
def MetaParse(MetaUrl):
    '''Function to parse out the URL response'''
    response = urllib.request.urlopen(MetaUrl)
    jsonRaw = response.read()
    jsonData = json.loads(jsonRaw)
    #return jsonData
    if jsonData['status'] == "OK":
        return (jsonData['pano_id'],jsonData['location']['lat'],jsonData['location']['lng'])
    else:
        return (None,None,None)
        
def GetStreetLL(Lat,Lon,Head,File,SaveLoc):
    '''Funciton to create the URL and give it to the street view API'''
    base = r"https://maps.googleapis.com/maps/api/streetview"
    size = r"?size=640x640&location="
    end = str(Lat) + "," + str(Lon) + "&heading=" + str(Head) + key
    MyUrl = base + size + end
    fi = File + ".jpg"
    MetaUrl = base + r"/metadata" + size + end
    #print MyUrl, MetaUrl #can check out image in browser to adjust size, fov to needs
    met_lis = list(MetaParse(MetaUrl))
    
    if (met_lis[0],Head) not in PrevImage:   #PrevImage is global list
        urllib.request.urlretrieve(MyUrl, os.path.join(SaveLoc,fi))
        met_lis.append(fi)
        PrevImage.append((met_lis[0],Head)) #append new Pano ID to list of images
    else:
        met_lis.append(None)
    return met_lis 

In [36]:
# Make some data containers to keep information returned from the API:
PrevImage = []        # Global list that has previous images sampled
points_returned = []  # list of all finalized points returned

# find if the start location has an assoicated street view, and it's coordinates:
temp = GetStreetLL(start_location[0],start_location[1],Head=heading,File='Test_origin',SaveLoc=DownLoc)
if temp[3] is not None:  # if this came back with a photo, then good, we will keep that photo, and...
    origy = temp[1]      # keep the y coordinate returned for that photo for later use
    origx = temp[2]      # keep the x coordinate returned for that photo for later use
    #points_returned.append([origy,origx])
    print('There is a photo here, saving to folder.')
# print the results of this:
print('temp list: '+str(temp))
print('previous image: '+str(PrevImage))
print('points returned: '+str(points_returned))

There is a photo here, saving to folder.
temp list: ['1YKdzRtdiST6TNqgtmQvzQ', 47.82944653171898, -119.1337098570937, 'Test_origin.jpg']
previous image: [('1YKdzRtdiST6TNqgtmQvzQ', 120)]
points returned: []


In [37]:
# make a point of displacement
def next_point_guess(lat,long,approx_disp,rough_direction):
    '''This function is used to make a guess at the next point along our path
    inputs:
    lat, long: input starting latitude and longitude (int)
    approx_disp: displacement in feet that we think we should go to the next point (int)
    rough_direction: direction in radial degrees we think the next point should be
    output:
    newlat, newlong: calculated new latitude and longitude'''
    disp_deg = round(approx_disp / (mi_to_ft * deg_to_mi),7)            # calculate the input displacement in feet
    y_disp = round(math.sin(math.pi * rough_direction /180)*disp_deg,7) # calculate the y component of displacement
    x_disp = round(math.cos(math.pi * rough_direction /180)*disp_deg,7) # calculate the x component of displacement
    newlat = lat + y_disp     # the new latitude 
    newlong = long + x_disp   # the new longitude
    return (newlat,newlong)   # values to return


In [38]:
# now lets guess where the next street view point might be..
nexty,nextx = next_point_guess(origy,origx,approx_disp,rough_direction)  # make a guess
temp = GetStreetLL(nexty,nextx,Head=heading,File='Test_second',SaveLoc=DownLoc) # get the street view at that point
if temp[3] is not None: # if we got a new point (the ID wasn't already in the list of previous points) then it worked...
    rety = temp[1] 
    retx = temp[2]
    #points_returned.append([rety,retx])
    print('We have the next point, saving.')
else:
    print('That was too close to the original, try a bit more distance')
# print the results of this:
print('temp list: '+str(temp))
print('previous image: '+str(PrevImage))
print('points returned: '+str(points_returned))

We have the next point, saving.
temp list: ['paZi5h8KnYUc2TR_u9FXXA', 47.82937046581372, -119.1337716766599, 'Test_second.jpg']
previous image: [('1YKdzRtdiST6TNqgtmQvzQ', 120), ('paZi5h8KnYUc2TR_u9FXXA', 120)]
points returned: []


In [39]:
# re-adjust the bearing and distance using a function:
def adjust_disp_bearing(origy,rety,origx,retx,rough_direction):
    '''Calculate the displacement and bearing from the previous 2 points'''
    y_diff = origy - rety
    x_diff = origx - retx
    disp = round(math.sqrt((y_diff ** 2)+(x_diff ** 2)) * mi_to_ft * deg_to_mi,0)
    new_direction = round(math.atan2(y_diff,x_diff) * 180 / math.pi,0)
    if rough_direction - new_direction > 90:
        new_direction = new_direction + 180
    return (disp,new_direction)
# run the function
approx_disp,new_direction = adjust_disp_bearing(origy,rety,origx,retx,rough_direction)
points_returned.append([rety,retx,approx_disp,new_direction])
# check what the new distance and bearing are
print('new distance is ' + str(approx_disp))
print('new bearing is '+str(new_direction))
print('points returned: '+str(points_returned))

new distance is 36.0
new bearing is 231.0
points returned: [[47.82937046581372, -119.1337716766599, 36.0, 231.0]]


In [40]:
# now let's see how far we are from the destination point- we can use the same function:
dis2dest,dest_direction = adjust_disp_bearing(nexty,end_location[0],nextx,end_location[1],rough_direction)
print('distance to destionation is about ' + str(dis2dest) + ' feet or ' + str(round(dis2dest/mi_to_ft,1)) + ' miles')
print('bearing is ' + str(dest_direction))

distance to destionation is about 7701.0 feet or 1.5 miles
bearing is 213.0


In [41]:
# ok, now we can put it all together into a loop that looks down the road for the next point, confirms or adjusts to try again...

counter = 0           # start the counter at 0, to see how many frames we collect in all
countermax = 100      # set a max of 100 to start with- don't want to wind up with a crazy Google API bill this month...
while (counter < countermax and dis2dest > 2000):
    print('The count is:' + str(counter))
    counter = counter + 1        # step up the counter
    origy = nexty                # make the last returned y the origin
    origx = nextx                # make the last returned x the origin
    nexty,nextx = next_point_guess(origy,origx,approx_disp,new_direction)   # calculate the next point along the road
    temp = GetStreetLL(nexty,nextx,Head=heading,File='file'+str(counter).zfill(5),SaveLoc=DownLoc)
    if temp[3] is not None:
        rety = temp[1]
        retx = temp[2]
        print('We have the next point, saving as ' + temp[3])
        approx_disp,new_direction = adjust_disp_bearing(origy,rety,origx,retx,rough_direction)
        points_returned.append([rety,retx,approx_disp,new_direction])
        reset = 1
    elif reset < 3:
        print('That was too close to the original, trying a bit more distance')
        approx_disp = approx_disp/2 + approx_disp
        reset = reset + 1
    else: 
        counter = 100
    print(str(approx_disp))
    print(str(new_direction))
    dis2dest,dest_direction = adjust_disp_bearing(nexty,end_location[0],nextx,end_location[1],rough_direction)
    print('distance to destionation is about ' + str(dis2dest) + ' feet or ' + str(dis2dest/mi_to_ft) + ' miles')
print('done')

The count is:0
We have the next point, saving as file00001.jpg
37.0
236.0
distance to destionation is about 7667.0 feet or 1.4520833333333334 miles
The count is:1
We have the next point, saving as file00002.jpg
38.0
235.0
distance to destionation is about 7633.0 feet or 1.4456439393939393 miles
The count is:2
We have the next point, saving as file00003.jpg
35.0
230.0
distance to destionation is about 7598.0 feet or 1.4390151515151515 miles
The count is:3
We have the next point, saving as file00004.jpg
34.0
225.0
distance to destionation is about 7564.0 feet or 1.4325757575757576 miles
The count is:4
We have the next point, saving as file00005.jpg
36.0
225.0
distance to destionation is about 7531.0 feet or 1.4263257575757575 miles
The count is:5
We have the next point, saving as file00006.jpg
40.0
230.0
distance to destionation is about 7496.0 feet or 1.4196969696969697 miles
The count is:6
We have the next point, saving as file00007.jpg
37.0
236.0
distance to destionation is about 7457

We have the next point, saving as file00057.jpg
38.0
225.0
distance to destionation is about 5705.0 feet or 1.0804924242424243 miles
The count is:57
We have the next point, saving as file00058.jpg
39.0
213.0
distance to destionation is about 5669.0 feet or 1.0736742424242425 miles
The count is:58
We have the next point, saving as file00059.jpg
40.0
209.0
distance to destionation is about 5630.0 feet or 1.066287878787879 miles
The count is:59
We have the next point, saving as file00060.jpg
39.0
216.0
distance to destionation is about 5590.0 feet or 1.058712121212121 miles
The count is:60
We have the next point, saving as file00061.jpg
39.0
225.0
distance to destionation is about 5551.0 feet or 1.0513257575757575 miles
The count is:61
We have the next point, saving as file00062.jpg
40.0
225.0
distance to destionation is about 5514.0 feet or 1.044318181818182 miles
The count is:62
We have the next point, saving as file00063.jpg
41.0
217.0
distance to destionation is about 5476.0 feet or 1

In [42]:
print('points returned:')
points_returned

points returned:


[[47.82937046581372, -119.1337716766599, 36.0, 231.0],
 [47.82929186733913, -119.1338367759438, 37.0, 236.0],
 [47.82921326886454, -119.1339018752277, 38.0, 235.0],
 [47.8291425302374, -119.1339604645831, 35.0, 230.0],
 [47.8290639317628, -119.134025563867, 34.0, 225.0],
 [47.82898533328821, -119.1340906631509, 36.0, 225.0],
 [47.82890673481361, -119.1341557624348, 40.0, 230.0],
 [47.82883599618648, -119.1342143517903, 37.0, 236.0],
 [47.8287573977119, -119.1342794510742, 35.0, 236.0],
 [47.8286787992373, -119.1343445503581, 35.0, 230.0],
 [47.8286056904439, -119.1344045779804, 35.0, 225.0],
 [47.82852674522195, -119.1344687389902, 37.0, 226.0],
 [47.8284478, -119.1345329, 39.0, 232.0],
 [47.82837521915164, -119.134593768387, 36.0, 236.0],
 [47.82830099072647, -119.134655771813, 33.0, 234.0],
 [47.82822242203735, -119.1347209495092, 34.0, 228.0],
 [47.82814385334822, -119.1347861272054, 38.0, 225.0],
 [47.82808885526583, -119.1348317515927, 30.0, 227.0],
 [47.82801028657671, -119.13489

In [36]:
print(str(rety)+','+str(retx))

47.82332936657829,-119.1403209984003


In [45]:
plotpoints = []
for i in range(len(points_returned)):
    readline = points_returned[i]
    plotpoints.append((readline[0],readline[1]))
#print(plotpoints)
symbols = gmaps.symbol_layer(plotpoints, fill_color='red', stroke_color='red')
fig.add_layer(symbols)
fig

Figure(layout=FigureLayout(height='420px'))